### Opening

In [3]:
import pandas as pd

In [ ]:
import os
import pandas as pd

folder = "/Users/luisfaria/Desktop/sEngineer/masters_SWEAI/2025-T2/T2-REM/assignments/Assessment2/sales"

# If in codespaces
# folder = "/workspaces/masters-swe-ai/2025-T2/T2-REM/assignments/Assessment2/sales"

# List to hold dataframes
dataframes = []

# Read all Excel files and append to list
for file in os.listdir(folder):
    if file.endswith(".xlsx"):
        filepath = os.path.join(folder, file)
        df = pd.read_excel(filepath)
        dataframes.append(df)

# Concatenate all dataframes into one
if dataframes:
    combined_df = pd.concat(dataframes, ignore_index=True)
    print("Data combined successfully.")
else:
    print("No Excel files found.")

# Optional: print the first few rows
combined_df.sample(5)

### Treating

In [5]:
df.columns

Index(['ID orçamento', 'Data orçamento', 'Status', 'Revenda', 'Data venda',
       'Mês venda', 'Hora venda', 'Unidade', 'Consultor', 'Avaliador',
       'Valor tabela', 'Valor desconto', 'Valor líquido', 'Grupo procedimento',
       'Procedimento', 'Quantidade', 'Valor tabela item',
       'Valor % desconto item', 'valor desconto item', 'Valor liquido item',
       'Fonte do cadastro do cliente', 'Nome cliente', 'CPF cliente',
       'ID cliente', 'Email do cliente', 'Telefone(s) do cliente',
       'Data nascimento cliente', 'Profissão cliente', 'Cortesia?'],
      dtype='object')

In [6]:
desired_columns = [
        'ID orçamento', 'Status','Data venda', 'Mês venda',
        'Unidade', 'Valor líquido', 'ID cliente']

df_filtered = combined_df[desired_columns]

# treating
df_filtered = df_filtered.loc[df_filtered['Status'] == 'Finalizado']
df_filtered['Valor líquido'] = df_filtered['Valor líquido'].fillna(0)
df_filtered.sort_values(by='Valor líquido', ascending=False)

# total sum of 'Valor líquido'
df_filtered['Valor líquido'].sum()



17719676.38

In [7]:
df_filtered['Data venda'] = pd.to_datetime(df_filtered['Data venda'])
df_filtered['Year'] = df_filtered['Data venda'].dt.year
df_filtered['Month'] = df_filtered['Data venda'].dt.month

### Checking

In [8]:
# convert 'Valor líquido' to float
df_filtered['Valor líquido'] = df_filtered['Valor líquido'].astype(float)


df_filtered.groupby(['Unidade', 'Month', 'Year'])['Valor líquido'].sum().unstack(fill_value=0)

Year                       2024      2025
Unidade        Month                     
ALPHAVILLE     1           0.00  76386.00
               2           0.00  82787.04
               3           0.00  60769.91
               12      92290.08      0.00
BELO HORIZONTE 1      111643.77      0.00
...                         ...       ...
TUCURUVI       12     152590.93      0.00
VILA MASCOTE   1           0.00  94511.00
               2           0.00  60550.99
               3           0.00  50164.01
               12      84607.97      0.00

[88 rows x 2 columns]

In [9]:
df_tucuruvi = df_filtered.loc[df_filtered['Unidade'] == 'TUCURUVI']
df_tucuruvi.groupby(['Unidade', 'Month', 'Year'])['Valor líquido'].sum().unstack(fill_value=0)

Year                 2024       2025
Unidade  Month                      
TUCURUVI 1      187160.08  164754.90
         2           0.00  155763.51
         3           0.00  160833.49
         12     152590.93       0.00

### Preparing to merge

In [12]:
df_filtered.columns

Index(['ID orçamento', 'Status', 'Data venda', 'Mês venda', 'Unidade',
       'Valor líquido', 'ID cliente', 'Year', 'Month', 'month'],
      dtype='object')

In [ ]:
df_filtered['month'] = df_filtered['Year'].astype(str) + '-' + df_filtered['Month'].astype(str).str.zfill(2)

,ID orçamento,Status,Data venda,Mês venda,Unidade,Valor líquido,ID cliente,Year,Month,month
0,403463,Finalizado,2025-01-03 12:12:43,Janeiro,LAPA,0.00,962784.0,2025,1,2025-01
1,412497,Finalizado,2025-01-10 18:01:45,Janeiro,JARDINS,0.00,562644.0,2025,1,2025-01
2,414983,Finalizado,2025-01-06 18:22:45,Janeiro,TATUAPÉ,597.04,1001222.0,2025,1,2025-01
3,415719,Finalizado,2025-01-09 13:42:05,Janeiro,TATUAPÉ,42.00,996194.0,2025,1,2025-01
4,421122,Finalizado,2025-01-10 10:28:18,Janeiro,TATUAPÉ,150.00,1005389.0,2025,1,2025-01
...,...,...,...,...,...,...,...,...,...,...
26844,435194,Finalizado,2025-02-28 21:23:21,Fevereiro,LONDRINA,199.00,466398.0,2025,2,2025-02
26845,435195,Finalizado,2025-02-28 21:29:02,Fevereiro,LONDRINA,100.00,121599.0,2025,2,2025-02
26846,435196,Finalizado,2025-02-28 21:54:30,Fevereiro,LONDRINA,96.00,950085.0,2025,2,2025-02
26847,435197,Finalizado,2025-02-28 22:26:49,Fevereiro,MOEMA,597.00,1023240.0,2025,2,2025-02


In [13]:
df_sales_summary = df_filtered.groupby(['month', 'Unidade']).agg(
    total_valor_liquido=('Valor líquido', 'sum'),
    count_of_sales=('ID orçamento', 'count')
).reset_index()

# Optional: Rename 'Unidade' to 'Store' to match NPS dataframe column name
df_sales_summary = df_sales_summary.rename(columns={'Unidade': 'Store'})

In [14]:
df_sales_summary

,month,Store,total_valor_liquido,count_of_sales
0,2024-01,BELO HORIZONTE,111643.77,182
1,2024-01,CAMPINAS,114131.31,246
2,2024-01,COPACABANA,225878.13,250
3,2024-01,INSIDE SALES,2350.03,2
4,2024-01,IPIRANGA,234789.84,305
...,...,...,...,...
99,2025-03,SÃO BERNARDO,93604.30,187
100,2025-03,TATUAPÉ,230494.01,280
101,2025-03,TIJUCA,263496.07,367
102,2025-03,TUCURUVI,160833.49,131
